<a href="https://colab.research.google.com/github/restrepo/anomaly/blob/main/Type_II_Dirac_Seesaw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solutions for effective Dirac neutrino masses from an Abelian Gauge Symmetry with massive chiral singlet fermions
For details see [arXiv:2108.05907](https://arxiv.org/pdf/2108.05907), Sec. 3.2

## Data Scheme
![img](https://raw.githubusercontent.com/restrepo/anomaly/main/schema.svg)

m=0

In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
pd.set_option('display.max_colwidth',500)

### Load full solutions

In [3]:
ds=pd.read_json('https://github.com/restrepo/anomaly/raw/main/solutions.json.gz')
ds.shape

(390074, 5)

## Filter two set of repeated charges
with one of them repeated 3 times
$$(\nu,\nu[,\nu],\psi_1,\psi_2,\cdots)$$


In [4]:
#at least one set of repeated numbers
dsp=ds[ds['solution'].apply(lambda l: len(l)-len(set(l))>0)].reset_index(drop=True)
dsp.shape

(280959, 5)

Some reordering

In [5]:
dsp['nmax']=dsp['solution'].apply(lambda l: map(abs,l)).apply(max)
dsp=dsp.sort_values(['n','nmax']).reset_index(drop=True)
dsp2=dsp.copy()
#Show the first and last one solutions
dsp#.iloc[[0,1,2,-1]]

,l,k,solution,gcd,n,nmax
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,5
1,"[-1, 1]","[-2, 0]","[1, -2, -3, 5, 5, -6]",1,6,6
2,"[0, 1]","[-1, -3, 1]","[1, 1, 5, -9, -9, 11]",2,6,11
3,"[1, -2]","[-4, 1]","[1, -4, -4, 9, 9, -11]",3,6,11
4,"[-2, -1]","[-2, -3]","[2, 2, 3, -8, -12, 13]",1,6,13
...,...,...,...,...,...,...
280954,"[-2, -3, -1, 3, 2]","[-1, 3, 1, -1, -3]","[1, 2, -10, 12, 19, 19, -23, -24, -30, -30, 32, 32]",2,12,32
280955,"[-1, -2, 0, 3, 2]","[-1, 3, 1, -1, -2]","[1, 1, 1, 1, 8, -12, -19, 20, 21, -23, -31, 32]",1,12,32
280956,"[-1, 2, -3, -2, -3]","[-1, 3, 2, -2, -1]","[1, 2, 2, 4, 5, -7, -7, -9, -11, 25, 27, -32]",2,12,32
280957,"[-1, 3, -2, 2, -2]","[-1, 3, 2, -1, -2]","[1, 3, 4, 8, -9, -9, -12, 19, 19, -28, -28, 32]",1,12,32


## 1. Check effective conditions

$$ \nu+m+\delta s=0$$

$$ \psi_i+\psi_j=|s|$$

In [6]:
def get_massless(l,s,rank=True):
    if len(l)==0:
        return []
    elif len(l)==1:
        mssls=[(l[0],l[0])]
    elif len(l)==2:
        mssls=[tuple(l)]
    else:
        mssls=set([x for x in itertools.permutations(l,2) if x[0]<=x[1] ])
    xs=[]
    massless=[] # zero eigenvalues when rank is not enough
    for x in mssls:
        if abs(x[0]+x[1])==abs(s):
            xs.append(x)  # ←→ xs=[ x for x in mssls if abs(x[0]+x[1])==abs(s) ] for rank=False
            #Check the matrix rank and store zero eigenvalues
            nr=l.count(x[0])-l.count(x[1])            
            if rank and nr>0 and abs(2*x[0])!=abs(s):
                massless=massless+ [ x[0] for i in range(nr)]
            elif rank and nr<0 and abs(2*x[1])!=abs(s):
                massless=massless+ [ x[1] for i in range(abs(nr))]
            #Check repetead massive

    #flatten list of lists into list
    massive=[x for sublist in xs for x in sublist]
    extra_massless=[]
    if rank:
        extra_massless=list(set([x for x in massive if massive.count(x)>1]))
        extra_massless=[x for x in extra_massless if abs(2*x)!=abs(s)]

    return list( set(l).difference(set(massive)) ) + massless + extra_massless  

def get_sp(massless):
    """
    For a list of numbers `x_i` find the best `sp` value such that
    all the elements of the list satisfy:
       |sp|=|x_i + x_j|
    * If the list is already empty return None
    * If not found `sp` returns 0
    """
    if len(massless)==0:
        return None
    elif len(massless)==1:
        drs=[(massless[0],massless[0])]
    elif len(massless)==2:
        drs=[tuple(massless)]
    else:
        drs=[x for x in itertools.permutations(massless,2) if x[0]<=x[1]]
    sps=set(abs(sum(x)) for x in drs)    
    for sp in sps:
        msv =([dr for dr in drs if abs(sum(dr))==sp])
        massive=set([x for sublist in msv for x in sublist])
        msl=([dr for dr in drs if abs(sum(dr))!=sp])
        massless=set( [x for sublist in msl for x in sublist])
        majorana=massless.difference(massive)
        majorana=[m for m in majorana if abs(2*m)!=sp]
        if not majorana:
            return sp
        else:
            return 0        

In [7]:
Type='TII-X'
#l=[1,-9,-9,-9,13,18,18,18,-19,-22]
#def conditions(l):
def condition(l,δ=1,rank=True):
#if True:   
    sls=[]
    ms=set([x for x in l if l.count(x)==3])
    if not ms:
        ms={0}
    νs=set([x for x in l if l.count(x)==2 or l.count(x)==3])
    mνs=[(m,ν) for m in ms for ν in νs if m!=ν]
    sold=0
    for mν in mνs:
        m=mν[0]
        ν=mν[1]
        s=(-m-ν)/δ
        if float(s)-int(s)==0:
            s=int(s)
        else:
            continue
        if abs(s)==abs(sold):
            continue
        else:
            sold=s
        ψs=[x for x in l if x!=ν and x!=m]
        msslss=get_massless(ψs,s,rank=rank)
        sp=get_sp(msslss)
        d={'m':m,'ν':ν,'s':s,'massless':msslss,'sp':sp,'δ':δ}
        sls.append(d)
    #sp=None    
    solutions=[d for d in sls if d.get('massless')==[]]
    if not solutions:
        solutions=[d for d in sls if d.get('sp')]    
    return solutions

l=[1, 2, -6, -6, -6, 8, 9, 9, -11]
assert condition(l)[0].get('massless')==[]
l=[1, 1, 1, -4, -4, 5]
assert condition(l)[0].get('sp')==10
l=[1,-9,-9,-9,13,18,18,18,-19,-22]
#m=0
assert condition(l)[0].get('sp')==18
l=[1, -2, -3, 5, 5, -6]
assert condition(l)[0].get('massless')==[]
#zero eigenvalues with repeated charges → -20
l=[13, 13, -16, -16, -20, -20, 23, 29, 29, 29, -32, -32]
assert condition(l,rank=False)[0]['massless']==[]
assert condition(l,rank=True)[0]['massless']==[-20]
#zero eigenvalues with non-repeated charges → -11,-17
l=[4, -7, -7, 10, -11, 14, 15, -17, 18, -21, -22, 24]
assert condition(l,rank=False)[0]['massless']==[]
assert len(condition(l,rank=True)[0]['massless'])==2

In [10]:
def unconditional_stability(row,ZN={6:(2,3),10:(2,5),15:(3,5)},DMs={},check_massless=True):
    '''
    According to appendix of arXiv:1911.05515, 
    generalized to any N
    
    '''
    if check_massless:
        sd=[d for d in row['effective'] if d.get('massless')==[]]
    else:
        sd=row['effective']
        
    for i in range(len(sd)):
        DMs={}
        d=sd[i]
        N=abs(d['s'])
        l=[n for n in row['solution'] if n not in [d['m'],d['ν']] ]
        #Check for non-hidden chiral fermions
        if 0 in np.array(l)%N:
            DMs={'None':tuple([])}
            continue
        DM=[x for x in itertools.permutations(l,2) 
               if x[0]<=x[1] and abs(sum(x))==N ]
        DM=DM+[(n,n) for n in l if abs(2*n)==N]
        #rank 0
        for dm in DM:
            if dm[0]!=dm[1] and l.count(dm[0])!=l.count(dm[1]) and abs(2*dm[0])!=N and abs(2*dm[0])!=N:
                DMs={'None':tuple([])}
        #missing massles → TODO: Check BUG
        if check_massless and np.setdiff1d(l,[item for sublist in DM for item in sublist]).size>0:
            DMs={'None':tuple([])}
        if not DMs.get('None')==():
            try:
                p,q=ZN[N]
            except KeyError:
                DMs={'None':tuple([])}
                continue
            ps=[n*p for n in range(1,N) if n*p<N]
            qs=[n*q for n in range(1,N) if n*q<N]
            ω=[ sorted([y%N for y in x]) for x in DM]
            DMF=['ψ' if np.setdiff1d(x,ps).size==0 else 'χ' 
                   if np.setdiff1d(x,qs).size==0 else 'None' for x in  ω ]
            DMs=dict(zip(DMF,DM))
            break
    return DMs

In [8]:
dsp['effective']=dsp['solution'].apply(condition)

In [9]:
dsp=dsp[dsp['effective'].apply(len)>0].reset_index(drop=True)

In [11]:
p=np.array( [2,3,5,7,11,13,17,19,23,29,31] )
q=p.copy()
zpq= [ pp*qq for pp in p for qq in q if pp<qq] 
Zpq=sorted( set( zpq ) )

In [12]:
pqs=[ (pp,qq) for pp in p for qq in q if pp<qq]
ZN=dict(zip(Zpq,pqs))

In [13]:
Zpq[:3]

[6, 10, 14]

In [14]:
dsp['DMs']=dsp.apply(lambda row: unconditional_stability(row,ZN,check_massless=False)  ,axis='columns')

In [16]:
dsp.shape

(17638, 8)

In [17]:
dspm=dsp[dsp['effective'].apply(lambda l: [d for d in l if not d.get('massless')]).apply(len)>0].reset_index(drop=True)
dspm[ dspm['effective'].apply(lambda l: 0 in [d.get('m') for d in l]) ].reset_index(drop=True)

,l,k,solution,gcd,n,nmax,effective,DMs
0,"[-1, 1]","[-2, 0]","[1, -2, -3, 5, 5, -6]",1,6,6,"[{'m': 0, 'ν': 5, 's': -5, 'massless': [], 'sp': None, 'δ': 1}]",{'None': ()}
1,"[1, 0]","[-1, -5, 0]","[2, -3, -10, 13, 13, -15]",2,6,15,"[{'m': 0, 'ν': 13, 's': -13, 'massless': [], 'sp': None, 'δ': 1}]",{'None': ()}
2,"[1, 0]","[-1, -4, 0]","[3, -5, -12, 17, 17, -20]",1,6,20,"[{'m': 0, 'ν': 17, 's': -17, 'massless': [], 'sp': None, 'δ': 1}]",{'None': ()}
3,"[1, 0]","[-1, -7, 0]","[3, -4, -21, 25, 25, -28]",2,6,28,"[{'m': 0, 'ν': 25, 's': -25, 'massless': [], 'sp': None, 'δ': 1}]",{'None': ()}
4,"[-3, 1, 2, -2]","[-4, -3, -4, -2, 0]","[1, -2, -2, 3, 3, -4, -4, 6, 6, -7]",1,10,7,"[{'m': 0, 'ν': 6, 's': -6, 'massless': [], 'sp': None, 'δ': 1}]","{'χ': (3, 3), 'ψ': (-4, -2), 'None': (-7, 1)}"
...,...,...,...,...,...,...,...,...
863,"[4, -6, -1, -5]","[-6, 5, -2, -6]","[4, -6, 7, -11, -14, 25, 25, -29, 31, -32]",30,10,32,"[{'m': 0, 'ν': 25, 's': -25, 'massless': [], 'sp': None, 'δ': 1}]",{'None': ()}
864,"[2, -4, 6, -6]","[-5, 6, -5, 0]","[2, -3, -5, 13, 14, -24, 25, -27, -27, 32]",24,10,32,"[{'m': 0, 'ν': -27, 's': 27, 'massless': [], 'sp': None, 'δ': 1}]",{'None': ()}
865,"[1, 6, 2, 1]","[-3, 6, 5, -4]","[5, 6, -11, -11, -13, -15, 21, 24, 26, -32]",12,10,32,"[{'m': 0, 'ν': -11, 's': 11, 'massless': [], 'sp': None, 'δ': 1}]",{'None': ()}
866,"[5, -5, -6, -2]","[-1, -6, 1, 2]","[1, -5, -5, 12, 19, -26, -26, 31, 31, -32]",58,10,32,"[{'m': 0, 'ν': 31, 's': -31, 'massless': [], 'sp': None, 'δ': 1}]",{'None': ()}


In [18]:
dspm[~dspm['effective'].apply(lambda l: 0 in [d.get('m') for d in l]) ].reset_index(drop=True)

,l,k,solution,gcd,n,nmax,effective,DMs
0,"[1, 2, 1, -1]","[-6, -3, -5, -3]","[1, 2, -6, -6, -6, 8, 9, 9, -11]",1,9,11,"[{'m': -6, 'ν': 9, 's': -3, 'massless': [], 'sp': None, 'δ': 1}]",{'None': ()}
1,"[-2, -1, 0]","[-1, -3, -5, -3]","[5, 6, 6, 6, -8, -9, -9, -10, 13]",2,9,13,"[{'m': 6, 'ν': -9, 's': 3, 'massless': [], 'sp': None, 'δ': 1}]",{'None': ()}
2,"[-2, -1, 0]","[-1, -4, -5, -4]","[1, -5, 8, 8, 8, -11, -12, -12, 15]",2,9,15,"[{'m': 8, 'ν': -12, 's': 4, 'massless': [], 'sp': None, 'δ': 1}]",{'None': ()}
3,"[3, -5, 0, -4]","[-2, -1, 0, -2]","[6, 6, 6, -9, -9, 20, -22, -23, 25]",6,9,25,"[{'m': 6, 'ν': -9, 's': 3, 'massless': [], 'sp': None, 'δ': 1}]",{'None': ()}
4,"[1, 0, 1, 0, -1]","[-2, -1, -2, -1, 0]","[1, 1, 1, -2, -2, -5, -5, 6, 6, 7, -8]",1,11,8,"[{'m': 1, 'ν': -2, 's': 1, 'massless': [], 'sp': None, 'δ': 1}]",{'None': ()}
...,...,...,...,...,...,...,...,...
133,"[2, 0, -3, 0, -1]","[-3, -2, -3, -2, -3]","[1, 6, -9, 14, 14, 14, 16, -21, -21, -21, -24, 31]",2,12,31,"[{'m': -21, 'ν': 14, 's': 7, 'massless': [], 'sp': None, 'δ': 1}]",{'None': ()}
134,"[1, -3, 1, -3, 1]","[-3, -1, -3, -1, -3]","[5, -15, -17, -17, -17, 21, 21, 27, 27, 27, -31, -31]",6,12,31,"[{'m': 27, 'ν': -17, 's': -10, 'massless': [], 'sp': None, 'δ': 1}, {'m': -17, 'ν': 27, 's': -10, 'massless': [], 'sp': None, 'δ': 1}]","{'χ': (5, 5), 'None': (-31, 21)}"
135,"[1, -3, 1, 2, 1]","[-3, -1, -3, -1, -3]","[5, 9, -13, -13, -13, -15, -19, 21, 23, 23, 23, -31]",4,12,31,"[{'m': -13, 'ν': 23, 's': -10, 'massless': [], 'sp': None, 'δ': 1}]","{'χ': (5, 5), 'None': (-31, 21)}"
136,"[-2, 2, -2, 1, -2]","[-1, -2, -1, 2, -1]","[2, -4, -4, -4, -7, 11, -14, -23, 25, 25, 25, -32]",3,12,32,"[{'m': 25, 'ν': -4, 's': -21, 'massless': [], 'sp': None, 'δ': 1}]","{'None': (-32, 11)}"


In [28]:
dspm[dspm['DMs'].apply(lambda d: 'ψ' in d.keys() and 'χ' in d.keys())].reset_index(drop=True)

,l,k,solution,gcd,n,nmax,effective,DMs
0,"[-3, 1, 2, -2]","[-4, -3, -4, -2, 0]","[1, -2, -2, 3, 3, -4, -4, 6, 6, -7]",1,10,7,"[{'m': 0, 'ν': 6, 's': -6, 'massless': [], 'sp': None, 'δ': 1}]","{'χ': (3, 3), 'ψ': (-4, -2), 'None': (-7, 1)}"
1,"[1, -1, 0, 2]","[-3, -2, 2, 0, 1]","[1, -2, -2, 3, 5, -6, -6, 8, 8, -9]",1,10,9,"[{'m': 0, 'ν': 8, 's': -8, 'massless': [], 'sp': None, 'δ': 1}, {'m': 0, 'ν': -6, 's': 6, 'massless': [], 'sp': None, 'δ': 1}]","{'None': (1, 5), 'ψ': (-2, 8), 'χ': (3, 3)}"
2,"[0, -2, 0, 2]","[-3, -2, -1, 0, -2]","[4, 4, 5, 6, 6, -9, -10, -10, -11, 15]",2,10,15,"[{'m': 0, 'ν': 4, 's': -4, 'massless': [], 'sp': None, 'δ': 1}, {'m': 0, 'ν': 6, 's': -6, 'massless': [], 'sp': None, 'δ': 1}]","{'χ': (-9, 15), 'ψ': (-10, 4), 'None': (-11, 5)}"
3,"[4, -1, 4, -3]","[-2, -3, -2, 1, -2]","[1, -2, -4, 7, 7, -10, -12, 14, 14, -15]",2,10,15,"[{'m': 0, 'ν': 14, 's': -14, 'massless': [], 'sp': None, 'δ': 1}]","{'χ': (7, 7), 'ψ': (-12, -2), 'None': (-15, 1)}"
4,"[-2, 0, -1, 0]","[-2, -1, 1, 0, -2]","[1, -2, -2, 7, 7, -14, -14, -15, 16, 16]",2,10,16,"[{'m': 0, 'ν': -14, 's': 14, 'massless': [], 'sp': None, 'δ': 1}]","{'ψ': (-2, 16), 'χ': (7, 7), 'None': (-15, 1)}"
5,"[-1, -4, -3, 0]","[-2, -1, 1, 3, -4]","[2, -3, -6, 7, 7, -8, -11, 14, 14, -16]",2,10,16,"[{'m': 0, 'ν': 14, 's': -14, 'massless': [], 'sp': None, 'δ': 1}]","{'χ': (7, 7), 'ψ': (-16, 2), 'None': (-11, -3)}"
6,"[-2, 0, 2, 1]","[-1, 1, 0, -3, 0]","[4, 4, -5, -7, -7, 14, 14, -18, -18, 19]",2,10,19,"[{'m': 0, 'ν': 14, 's': -14, 'massless': [], 'sp': None, 'δ': 1}]","{'None': (-5, 19), 'χ': (-7, -7), 'ψ': (-18, 4)}"
7,"[1, 0, 1, 0]","[-1, 0, -4, 2, -3]","[1, -4, 5, -6, -6, 10, -14, 15, 20, -21]",2,10,21,"[{'m': 0, 'ν': -6, 's': 6, 'massless': [], 'sp': None, 'δ': 1}]","{'None': (1, 5), 'ψ': (-14, 20), 'χ': (-21, 15)}"
8,"[2, 0, -2, -1]","[-3, -2, 0, -1, 0]","[2, -3, -6, 7, 12, -14, -14, 17, 20, -21]",4,10,21,"[{'m': 0, 'ν': -14, 's': 14, 'massless': [], 'sp': None, 'δ': 1}]","{'ψ': (-6, 20), 'None': (-3, 17), 'χ': (7, 7)}"
9,"[2, 0, -2, -1]","[-3, -2, 0, 1, 0]","[4, -5, 6, -7, -9, 14, 14, -18, -20, 21]",4,10,21,"[{'m': 0, 'ν': 14, 's': -14, 'massless': [], 'sp': None, 'δ': 1}]","{'χ': (-7, -7), 'None': (-9, -5), 'ψ': (-20, 6)}"


$\delta=2$

In [20]:
dsp2.shape

(280959, 6)

In [21]:
dsp2['effective']=dsp2['solution'].apply(lambda l:condition(l,δ=2))

In [22]:
dsp2=dsp2[dsp2['effective'].apply(len)>0].reset_index(drop=True)

In [23]:
dsp2.shape

(6991, 7)

In [24]:
dspm2=dsp2[dsp2['effective'].apply(lambda l: [d for d in l if not d.get('massless')]).apply(len)>0].reset_index(drop=True)
dspm2[ dspm2['effective'].apply(lambda l: 0 in [d.get('m') for d in l]) ].reset_index(drop=True)

,l,k,solution,gcd,n,nmax,effective
0,"[-2, -1, 1, 2]","[-2, -1, 0, 1]","[2, 2, 4, 4, -7, -7, -9, -9, 10, 10]",2,10,10,"[{'m': 0, 'ν': 10, 's': -5, 'massless': [], 'sp': None, 'δ': 2}]"
1,"[-1, 3, 1, -1]","[-3, -1, -4, 1, -1]","[1, 1, -3, 4, 4, -7, 8, -10, -10, 12]",1,10,12,"[{'m': 0, 'ν': -10, 's': 5, 'massless': [], 'sp': None, 'δ': 2}]"
2,"[0, 3, -2, 3]","[-2, -1, 0, -1, 2]","[1, 6, 7, 7, -10, -15, -16, -16, 18, 18]",2,10,18,"[{'m': 0, 'ν': 18, 's': -9, 'massless': [], 'sp': None, 'δ': 2}]"
3,"[2, -3, 0, 2]","[-1, -2, 0, -1, -2]","[3, -5, 6, -8, -8, 14, 17, 17, -18, -18]",2,10,18,"[{'m': 0, 'ν': -18, 's': 9, 'massless': [], 'sp': None, 'δ': 2}]"
4,"[-2, 3, 0, -2]","[-1, -2, 0, -1, -2]","[2, 2, 6, 10, -11, -11, -15, 18, 18, -19]",2,10,19,"[{'m': 0, 'ν': 18, 's': -9, 'massless': [], 'sp': None, 'δ': 2}]"
5,"[2, 1, 0, -1]","[-2, -3, -1, -2, 0]","[1, -3, -6, 7, -10, 11, -16, 18, 18, -20]",2,10,20,"[{'m': 0, 'ν': 18, 's': -9, 'massless': [], 'sp': None, 'δ': 2}]"
6,"[2, 1, 0, -1]","[-2, -3, -1, 2, 0]","[4, 4, 5, 5, -7, -12, 16, -18, -18, 21]",2,10,21,"[{'m': 0, 'ν': -18, 's': 9, 'massless': [], 'sp': None, 'δ': 2}]"
7,"[-1, -2, 2, 1]","[-3, 2, 0, -2]","[1, 1, 4, -10, -10, 12, -13, 18, 18, -21]",2,10,21,"[{'m': 0, 'ν': 18, 's': -9, 'massless': [], 'sp': None, 'δ': 2}]"
8,"[2, 1, 0, -1]","[-2, -3, 1, -2, 0]","[2, 3, -5, 6, 7, -13, 14, -18, -18, 22]",2,10,22,"[{'m': 0, 'ν': -18, 's': 9, 'massless': [], 'sp': None, 'δ': 2}]"
9,"[-2, -1, 0, -2]","[-1, -4, 0, -1, 2]","[5, 7, 7, 8, -16, -18, -18, -19, 22, 22]",2,10,22,"[{'m': 0, 'ν': 22, 's': -11, 'massless': [], 'sp': None, 'δ': 2}]"


In [25]:
dspm2[~dspm2['effective'].apply(lambda l: 0 in [d.get('m') for d in l]) ].reset_index(drop=True)

,l,k,solution,gcd,n,nmax,effective
0,"[-3, -1]","[-2, -3, 1]","[3, 3, 3, -5, -5, -7, 8]",1,7,8,"[{'m': 3, 'ν': -5, 's': 1, 'massless': [], 'sp': None, 'δ': 2}]"
1,"[3, -4, 5]","[-4, -3, 1, -3]","[1, -3, 8, 8, 8, -12, -12, -17, 19]",4,9,19,"[{'m': 8, 'ν': -12, 's': 2, 'massless': [], 'sp': None, 'δ': 2}]"
2,"[-1, 4, 2]","[-3, -8, 3, 4]","[8, 8, 8, -12, -12, 15, -17, -23, 25]",10,9,25,"[{'m': 8, 'ν': -12, 's': 2, 'massless': [], 'sp': None, 'δ': 2}]"
3,"[-1, 1, 0, 1]","[-2, -3, 2, 0, 1]","[1, -2, -3, -3, -3, 5, 5, -7, 8, 10, -11]",1,11,11,"[{'m': -3, 'ν': 5, 's': -1, 'massless': [], 'sp': None, 'δ': 2}]"
4,"[-1, -3, 0, -2, 1]","[-1, -2, 0, 3, -3]","[3, 3, 3, -4, -4, 7, 7, -8, -9, -9, 11]",1,11,11,"[{'m': 3, 'ν': -9, 's': 3, 'massless': [], 'sp': None, 'δ': 2}]"
5,"[-2, -1, -2, 0]","[-2, -3, 1, -1, 1]","[2, -3, 5, 5, 5, -7, -7, 8, -9, -13, 14]",3,11,14,"[{'m': 5, 'ν': -7, 's': 1, 'massless': [], 'sp': None, 'δ': 2}]"
6,"[-3, -1, -3, 0, 1]","[-3, -2, 0, -1, 0]","[1, -2, -3, -3, -3, 5, 5, -12, 13, 14, -15]",5,11,15,"[{'m': -3, 'ν': 5, 's': -1, 'massless': [], 'sp': None, 'δ': 2}]"
7,"[-1, -2, 0, -3, -2]","[-1, 3, 1, 0, 2]","[3, -4, 5, 5, 5, -7, -7, 12, -13, -16, 17]",2,11,17,"[{'m': 5, 'ν': -7, 's': 1, 'massless': [], 'sp': None, 'δ': 2}]"
8,"[2, -3, -2, 1]","[-2, -1, 0, -1, -4]","[5, -6, -6, -6, -7, -9, 10, 10, 11, 16, -18]",2,11,18,"[{'m': -6, 'ν': 10, 's': -2, 'massless': [], 'sp': None, 'δ': 2}]"
9,"[-2, -3, 0, -2, -1]","[-3, -1, 0, -3, 3]","[1, -4, 9, 9, 9, -10, 13, -15, -15, -18, 21]",8,11,21,"[{'m': 9, 'ν': -15, 's': 3, 'massless': [], 'sp': None, 'δ': 2}]"


In [29]:
dspm2['DMs']=dspm2.apply(lambda row: unconditional_stability(row,ZN,check_massless=False)  ,axis='columns')

In [30]:
dspm2[dspm2['DMs'].apply(lambda d: 'ψ' in d.keys() and 'χ' in d.keys())].reset_index(drop=True)

,l,k,solution,gcd,n,nmax,effective,DMs
